In [ ]:
#mu_hat 이랑 sigma_hat 구하는 코드

import pandas as pd
import numpy as np

path=r"/content/MD1(최종).csv"
df=pd.read_csv(path)

In [ ]:
DATE_COL="계약일"
PRICE_COL="거래금액(만원)"
AREA_COL="전용면적(㎡)"

In [ ]:
# 1) 전처리: 타입/결측/음수 제거
df = df.copy()
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df[PRICE_COL] = pd.to_numeric(df[PRICE_COL], errors="coerce")
df[AREA_COL] = pd.to_numeric(df[AREA_COL], errors="coerce")

df = df.dropna(subset=[DATE_COL, PRICE_COL, AREA_COL])
df = df[(df[PRICE_COL] > 0) & (df[AREA_COL] > 0)]

In [ ]:
# 2) 월별 단위면적가격(unit_price) 생성
df["ym"] = df[DATE_COL].dt.to_period("M").dt.to_timestamp()
df["unit_price"] = df[PRICE_COL] / df[AREA_COL]   # 단위: (만원/㎡)

In [ ]:
# 3) 월별 가격지수 P_t 생성 (median) + 월별 표본수 n
monthly = (
    df.groupby("ym")["unit_price"]
      .agg(n="size", P="median")
      .sort_index()
)

MIN_N = 10

In [ ]:
# 4) 표본수 부족 월은 제거(가장 안전/단순)
monthly_A = monthly[monthly["n"] >= MIN_N].copy()
monthly_A["r"] = np.log(monthly_A["P"] / monthly_A["P"].shift(1))
rA = monthly_A["r"].dropna()

mu_hat_A = rA.mean() * 12
sigma_hat_A = rA.std(ddof=1) * np.sqrt(12)

print("mu_hat (annualized):", mu_hat_A)
print("sigma_hat (annualized):", sigma_hat_A)

mu_hat (annualized): -0.007366224091615892
sigma_hat (annualized): 0.3133634101269932
